# Batch Analysis for hawk/dove simulation with multiple risk attitudes, no adjustment

- What % of time do agents play Hawk, by risk attitude? risk-inclined (R=1) and risk-avoidant (R8) agents play Hawk?
- Cumulative wealth analysis by risk attitude

To track how often agents play Hawk, we need to collect data for every round.

This data was generated by running:
```console
simulatingrisk/hawkdovemulti/batch_run.py --params no_adjustment --agent-data --collect-data every_round
```

Each row in the data file represents one round for each agent.

In [32]:
import polars as pl


# df = pd.read_csv("../../data/hawkdovemulti/2025-07-22T170057_747737_agent.csv")
# df = pd.read_csv("../../data/hawkdovemulti/2025-07-23T135510_859267_agent.csv")
# which batch run data to use; use variable to ensure we use matching agent and model data
batch_run_date = "2025-07-24T120337_924060"

# load agent data, drop unneeded columns, and add numeric value 1 for played hawk, 0 for played dove

df = (
    pl.read_csv(f"../../data/hawkdovemulti/{batch_run_date}_agent.csv")
        .drop("risk_level_changed")  # drop risk_level_changed; not relevant here (no adjustment = no changes)
        .rename({'risk_level': 'risk_attitude'})  # code still uses risk_level internally; relabel as risk attitude
        .with_columns(
            # add a numeric field to turn choice of play to 1/0 hawk, for aggregation            
            played_hawk=pl.when(pl.col("choice").eq("hawk")).then(1).otherwise(0)
        )
)
df.head()

RunId,iteration,Step,AgentID,risk_attitude,choice,points,played_hawk
i64,i64,i64,i64,i64,str,i64,i32
1,1,1,0,3,"""dove""",12,0
1,1,1,1,8,"""dove""",12,0
1,1,1,2,8,"""hawk""",12,1
1,1,1,3,4,"""hawk""",18,1
1,1,1,4,8,"""hawk""",15,1


## Percent of the time agents play Hawk, by risk attitude

What % of time do risk-inclined (R=1) and risk-avoidant (R8) agents play Hawk?

- Guess from observation is is >90% for R=1, <10% for R8, but we want to have statistics for this: for X trials, how many of them does R1 play Hawk more than 90% of the time?
- Also useful to have statistics e.g. R=2 played Hawk between 80-90% of the time, or whatever the result is.


In [2]:
# each row in the data frame is a play by an agent on the grid
# group by risk level, then:
# - count the number of rows (= total number of plays)
# - sum the played_hawk field (= number of times played hawk)
# - calculate percent of turns played hawk

hawk_by_risk_attitude = (
    df.group_by("risk_attitude")
        .agg(n_plays=pl.col("played_hawk").count(), n_plays_hawk=pl.col("played_hawk").sum())
    .with_columns(
        pct_plays_hawk=pl.col("n_plays_hawk").truediv(pl.col("n_plays")).mul(100)
    )
)

hawk_by_risk_attitude

risk_attitude,n_plays,n_plays_hawk,pct_plays_hawk
i64,u32,i32,f64
0,13280811,13101490,98.649774
9,13146745,178159,1.355157
3,15198831,10693764,70.359122
6,15209556,4506272,29.6279
7,13058019,1725094,13.210993
4,17567250,10583899,60.247899
1,12726041,11838227,93.023643
5,17605297,6979686,39.645375
8,12692401,882273,6.951191


In [12]:
# output core fields as nicely styled table

(hawk_by_risk_attitude
    .select("risk_attitude", "pct_plays_hawk")
    .sort("risk_attitude")
    .rename({"risk_attitude": "Risk Attitude", "pct_plays_hawk": "% plays Hawk"})
    .style.tab_header(title="% of time agents play Hawk by Risk Attitude")
    .fmt_number("% plays Hawk", decimals=1)
)

GT(_tbl_data=shape: (10, 2)
┌───────────────┬──────────────┐
│ Risk Attitude ┆ % plays Hawk │
│ ---           ┆ ---          │
│ i64           ┆ f64          │
╞═══════════════╪══════════════╡
│ 0             ┆ 98.649774    │
│ 1             ┆ 93.023643    │
│ 2             ┆ 86.810279    │
│ 3             ┆ 70.359122    │
│ 4             ┆ 60.247899    │
│ 5             ┆ 39.645375    │
│ 6             ┆ 29.6279      │
│ 7             ┆ 13.210993    │
│ 8             ┆ 6.951191     │
│ 9             ┆ 1.355157     │
└───────────────┴──────────────┘, _body=<great_tables._gt_data.Body object at 0x339c55e20>, _boxhead=Boxhead([ColInfo(var='Risk Attitude', type=<ColInfoTypeEnum.default: 1>, column_label='Risk Attitude', column_align='right', column_width=None), ColInfo(var='% plays Hawk', type=<ColInfoTypeEnum.default: 1>, column_label='% plays Hawk', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x339b77590>, _spanners=Spanners([]), _heading=Heading(title='% of time agents play Hawk by Risk Attitude', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x339c56ff0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x339c56ea0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', v

In [14]:
import altair as alt

alt.Chart(hawk_by_risk_attitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_plays_hawk", title="% time plays Hawk")
).properties(title="% of time agents play Hawk")

alt.Chart(...)

Context: how many runs are these numbers drawn from?



In [30]:
total_unique_runs = len(df["RunId"].unique())
total_iterations = len(df["iteration"].unique())
n_combinations = int(total_unique_runs / total_iterations)

# Step is the round count indicator
longest_run = df["Step"].max()    # highest across all runs
# average of max value for each run. Group by run, get max Step, average —> returns a dataframe, get first Step value
average_run = df.group_by("RunId").agg(pl.col("Step").max()).mean()["Step"].first()

print(f"""{total_unique_runs:,} total unique runs; {total_iterations} iterations of {n_combinations} different parameter combinations.

Longest run: {longest_run} steps
Average run: {average_run:.1f} steps
""")


13,500 total unique runs; 100 iterations of 135 different parameter combinations.

Longest run: 109 steps
Average run: 44.9 steps



### Analysis filtered by simulation parameters

In [31]:
# identify the last round of each run
# for both wealth analysis and model parameters, we want to look at the last round (Step) of each run

last_round_df = df.group_by("RunId").agg(pl.col("Step").max())
last_round_df.head()

RunId,Step
i64,i64
3296,31
268,31
1066,31
2632,31
7869,52


In [35]:
# load model data and filter to last round for each run
full_model_df = pl.read_csv(f"../../data/hawkdovemulti/{batch_run_date}_model.csv")
model_df = last_round_df.join(full_model_df, on=['RunId', 'Step'], how="left")
# limit to only those fields that are needed for our analysis
model_df = model_df.select("RunId", "iteration", "risk_distribution", "play_neighborhood", "observed_neighborhood", "grid_size")
model_df.head()

RunId,iteration,risk_distribution,play_neighborhood,observed_neighborhood,grid_size
i64,i64,str,i64,i64,i64
3296,96,"""normal""",8,24,25
268,68,"""uniform""",8,8,25
1066,66,"""uniform""",24,8,10
2632,32,"""uniform""",4,4,25
7869,69,"""skewed left""",4,4,5


In [38]:
print(f"""Simulation parameters:

Grid size: {', '.join(str(n) for n in sorted(model_df["grid_size"].unique()))}
Iniital risk distribution: {', '.join(str(n) for n in sorted(model_df["risk_distribution"].unique()))}
Play neighborhood size: {', '.join(str(n) for n in sorted(model_df["play_neighborhood"].unique()))}
Observed neighborhood sized: {', '.join(str(n) for n in sorted(model_df["observed_neighborhood"].unique()))}

""")

Simulation parameters:

Grid size: 5, 10, 25
Iniital risk distribution: bimodal, normal, skewed left, skewed right, uniform
Play neighborhood size: 4, 8, 24
Observed neighborhood sized: 4, 8, 24




In [39]:
# join agent data with model data so we can filter by starting parameters
agent_df_params = df.join(model_df, on=["RunId", "iteration"], how="left")

#### Grid size


In [41]:
hawk_by_gridsize_riskattitude = (
    agent_df_params.group_by("grid_size", "risk_attitude")
         .agg(n_plays=pl.col("played_hawk").count(), n_plays_hawk=pl.col("played_hawk").sum())
    .with_columns(
        pct_plays_hawk=pl.col("n_plays_hawk").truediv(pl.col("n_plays")).mul(100)
    )
)

alt.Chart(hawk_by_gridsize_riskattitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_plays_hawk", title="% time plays Hawk")
).facet(column=alt.Column("grid_size", title="Grid Size")).properties(title="% of time agents play Hawk")


alt.FacetChart(...)

#### Play neighborhood


In [42]:
hawk_by_playnhood_riskattitude = (
    agent_df_params.group_by("play_neighborhood", "risk_attitude")
         .agg(n_plays=pl.col("played_hawk").count(), n_plays_hawk=pl.col("played_hawk").sum())
    .with_columns(
        pct_plays_hawk=pl.col("n_plays_hawk").truediv(pl.col("n_plays")).mul(100)
    )
)

alt.Chart(hawk_by_playnhood_riskattitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_plays_hawk", title="% time plays Hawk")
).facet(column=alt.Column("play_neighborhood", title="Play Neighborhood")).properties(title="% of time agents play Hawk")


alt.FacetChart(...)

#### Observed neighborhood


In [44]:
hawk_by_obsnhood_riskattitude = (
    agent_df_params.group_by("observed_neighborhood", "risk_attitude")
         .agg(n_plays=pl.col("played_hawk").count(), n_plays_hawk=pl.col("played_hawk").sum())
    .with_columns(
        pct_plays_hawk=pl.col("n_plays_hawk").truediv(pl.col("n_plays")).mul(100)
    )
)

alt.Chart(hawk_by_obsnhood_riskattitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_plays_hawk", title="% time plays Hawk")
).facet(column=alt.Column("observed_neighborhood", title="Observed Neighborhood")).properties(title="% of time agents play Hawk")


alt.FacetChart(...)

#### Initial risk distribution


In [47]:
hawk_by_initialdist_riskattitude = (
    agent_df_params.group_by("risk_distribution", "risk_attitude")
         .agg(n_plays=pl.col("played_hawk").count(), n_plays_hawk=pl.col("played_hawk").sum())
    .with_columns(
        pct_plays_hawk=pl.col("n_plays_hawk").truediv(pl.col("n_plays")).mul(100)
    )
)

alt.Chart(hawk_by_initialdist_riskattitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_plays_hawk", title="% time plays Hawk")
).facet(facet=alt.Facet("risk_distribution", title="Initial Risk Distribution"), columns=3
).properties(title="% of time agents play Hawk")


alt.FacetChart(...)

## Cumulative Wealth analysis


- mean and quartiles for wealth by R
- does mean vary between Rs or is it roughly the same?
  - quartiles look different, but need a statistic; esp. compare lower-R quartile to higher-R quartile
     - expect/hope that lower quartile is higher for R1 than R8, higher quartile is higher for R8 than R1
     - what's going on in the middle?


-----

Because our simulations run for different lengths before stopping, and because our simulations include a range of different play neighborhoods (which affects total payoff), we scale points (wealth) by play neighborhood and simulation run length for comparison across simulations.


In [98]:
# combine the last round dataframe and full agent dataframe to get just the last round
agents_last_round_df = (
    last_round_df.join(df, on=['RunId', 'Step'], how="left")
        # join on model parameters, for filtering and scaling by play neighborhood
        .join(model_df, on=["RunId", "iteration"])
        .with_columns(
            # calculate a scaled points value so we can compare across runs with different length and play neighborhood
            scaled_points=pl.col("points").truediv(pl.col("play_neighborhood")).truediv(pl.col("Step")).mul(100)
        )
    
)

# hard to compare points across runs, since it depends on how long the simulation ran; scale points by number of runs
# agents_last_round_df['scaled_points'] = agents_last_round_df.apply(lambda x: (x['points'] / x['Step'])*10, axis=1)

# merge with model parameters, for filtering
# agents_last_round_df = pd.merge(agents_last_round_df, model_df, on=["RunId", "iteration"])

agents_last_round_df.head(10)

RunId,Step,iteration,AgentID,risk_attitude,choice,points,played_hawk,risk_distribution,play_neighborhood,observed_neighborhood,grid_size,scaled_points
i64,i64,i64,i64,i64,str,i64,i32,str,i64,i64,i64,f64
3296,31,96,0,4,"""hawk""",265,1,"""normal""",8,24,25,106.854839
3296,31,96,1,4,"""dove""",257,0,"""normal""",8,24,25,103.629032
3296,31,96,2,6,"""hawk""",280,1,"""normal""",8,24,25,112.903226
3296,31,96,3,6,"""hawk""",342,1,"""normal""",8,24,25,137.903226
3296,31,96,4,4,"""dove""",290,0,"""normal""",8,24,25,116.935484
3296,31,96,5,4,"""hawk""",332,1,"""normal""",8,24,25,133.870968
3296,31,96,6,2,"""hawk""",359,1,"""normal""",8,24,25,144.758065
3296,31,96,7,4,"""hawk""",296,1,"""normal""",8,24,25,119.354839
3296,31,96,8,2,"""hawk""",318,1,"""normal""",8,24,25,128.225806


First plot unscaled wealth distribution by risk attitude.

In [119]:
# our data has a lot of rows; enable vegafusion so altair can calculate quartiles for us
alt.data_transformers.enable("vegafusion")

alt.Chart(agents_last_round_df).mark_boxplot(extent="min-max").encode(
   x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.X("points", title="Wealth "),
).properties(title="Cumulative Wealth (unscaled) by Risk Attitude")

alt.Chart(...)

Now plot scaled wealth distribution by risk attitude.

In [111]:
wealthchart_title = alt.TitleParams(
    "Cumulative Wealth by Risk Attitude",
    subtitle=["Wealth scaled by simulation length and play neighborhood"]
)

alt.Chart(agents_last_round_df, title=wealthchart_title).mark_boxplot(extent="min-max").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.X("scaled_points", title="Wealth "),
) 

alt.Chart(...)

In [126]:
# Wanted to try a violin plot for comparison, but Altair/Vega can't handle the scale of our data,
# and I couldn't find an example of how to precompute the values to feed into an Altair mark area chart.


For comparison's sake, plot wealth at round 31 across all simulations, scaled only by play neighborhood.

This distribution looks similar to the scaled wealth plot.    

In [104]:
# what if we look at wealth at round 31 across simulations?

agents_round31_df = df.filter(pl.col("Step").eq(31)).join(model_df, on=["RunId", "iteration"]).with_columns(
    scaled_points=pl.col("points").truediv(pl.col("play_neighborhood")).mul(8)
)

alt.Chart(agents_round31_df).mark_boxplot(extent="min-max").encode(
   x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.X("scaled_points", title="Wealth"),
).properties(title="Cumulative wealth at round 31 across runs")

alt.Chart(...)

Calculate quartiles and other statistics, and output as a table for reference.

Calculate quartiles and other statistics and output as a table, for reference.

In [105]:
# Altair boxplot is calculating quartiles for us, but we can calculate them directly as well

wealth_by_risk_attitude = agents_last_round_df.group_by("risk_attitude").agg(
    min=pl.col("scaled_points").min(), 
    max=pl.col("scaled_points").max(),
    mean=pl.col("scaled_points").mean(), 
    Q1=pl.col("scaled_points").quantile(0.25),
    Q2=pl.col("scaled_points").quantile(0.5),
    Q3=pl.col("scaled_points").quantile(0.75),
).sort("risk_attitude")

In [107]:
# output wealth by risk attitude as nicely styled table

(wealth_by_risk_attitude
    .rename({"risk_attitude": "Risk Attitude"})
    .style.tab_header(title="Cumulative Wealth by Risk Attitude", subtitle="Wealth scaled by simulation length and play neighborhood")
    .fmt_number(decimals=1)
    .fmt_number("Risk Attitude", decimals=0)
)

GT(_tbl_data=shape: (10, 7)
┌───────────────┬───────────┬────────────┬────────────┬────────────┬────────────┬────────────┐
│ Risk Attitude ┆ min       ┆ max        ┆ mean       ┆ Q1         ┆ Q2         ┆ Q3         │
│ ---           ┆ ---       ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│ i64           ┆ f64       ┆ f64        ┆ f64        ┆ f64        ┆ f64        ┆ f64        │
╞═══════════════╪═══════════╪════════════╪════════════╪════════════╪════════════╪════════════╡
│ 0             ┆ 0.0       ┆ 300.0      ┆ 148.685253 ┆ 114.919355 ┆ 149.691358 ┆ 174.596774 │
│ 1             ┆ 0.0       ┆ 300.0      ┆ 144.020105 ┆ 113.206215 ┆ 147.580645 ┆ 167.79661  │
│ 2             ┆ 0.0       ┆ 300.0      ┆ 145.429545 ┆ 112.5      ┆ 147.580645 ┆ 170.564516 │
│ 3             ┆ 0.0       ┆ 300.0      ┆ 143.420712 ┆ 110.849057 ┆ 137.5      ┆ 165.957447 │
│ 4             ┆ 0.0       ┆ 300.0      ┆ 141.715225 ┆ 109.946237 ┆ 134.659091 ┆ 157.258065 │
│ 5             ┆ 1.694915  ┆ 300.0      ┆ 139.42156  ┆ 114.344262 ┆ 137.5      ┆ 150.806452 │
│ 6             ┆ 43.220339 ┆ 300.0      ┆ 143.347455 ┆ 125.403226 ┆ 143.025362 ┆ 154.301075 │
│ 7             ┆ 64.705882 ┆ 300.0      ┆ 147.048369 ┆ 137.037037 ┆ 149.327957 ┆ 158.064516 │
│ 8             ┆ 87.096774 ┆ 300.0      ┆ 151.041036 ┆ 140.254237 ┆ 150.403226 ┆ 161.693548 │
│ 9             ┆ 96.774194 ┆ 203.225806 ┆ 150.287365 ┆ 141.397849 ┆ 150.0      ┆ 160.887097 │
└───────────────┴───────────┴────────────┴────────────┴────────────┴────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x36803a690>, _boxhead=Boxhead([ColInfo(var='Risk Attitude', type=<ColInfoTypeEnum.default: 1>, column_label='Risk Attitude', column_align='right', column_width=None), ColInfo(var='min', type=<ColInfoTypeEnum.default: 1>, column_label='min', column_align='right', column_width=None), ColInfo(var='max', type=<ColInfoTypeEnum.default: 1>, column_label='max', column_align='right', column_width=None), ColInfo(var='mean', type=<ColInfoTypeEnum.default: 1>, column_label='mean', column_align='right', column_width=None), ColInfo(var='Q1', type=<ColInfoTypeEnum.default: 1>, column_label='Q1', column_align='right', column_width=None), ColInfo(var='Q2', type=<ColInfoTypeEnum.default: 1>, column_label='Q2', column_align='right', column_width=None), ColInfo(var='Q3', type=<ColInfoTypeEnum.default: 1>, column_label='Q3', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x6d86f1700>, _spanners=Spanners([]), _heading=Heading(title='Cumulative Wealth by Risk Attitude', subtitle='Wealth scaled by simulation length and play neighborhood', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x368052150>, _formats=[<great_tables._gt_data.FormatInfo object at 0x6d86f13a0>, <great_tables._gt_data.FormatInfo object at 0x6d86f32c0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=Opti

### Analysis filtered by simulation parameters

How does the wealth distribution vary based on other starting parameters?

#### Grid size

In [113]:
alt.Chart(agents_last_round_df).mark_boxplot(extent="min-max").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.X("scaled_points", title="Wealth"),
).facet(column=alt.Column("grid_size", title="Grid Size")).properties(title=wealthchart_title)

alt.FacetChart(...)

#### Play neighborhood

In [115]:
alt.Chart(agents_last_round_df).mark_boxplot(extent="min-max").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.X("scaled_points", title="Wealth"),
).facet(column=alt.Column("play_neighborhood", title="Play Neighborhood")
).properties(title=wealthchart_title)

alt.FacetChart(...)

#### Observed neighborhood

In [118]:
alt.Chart(agents_last_round_df).mark_boxplot(extent="min-max").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.X("scaled_points", title="Wealth"),
).facet(column=alt.Column("observed_neighborhood", title="Observed Neighborhood")
).properties(title=wealthchart_title)

alt.FacetChart(...)

#### Initial risk distribution

In [117]:
alt.Chart(agents_last_round_df).mark_boxplot(extent="min-max").encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.X("scaled_points", title="Wealth"),
).facet(facet=alt.Facet("risk_distribution", title="Initial Risk Distribution"), columns=3
).properties(title=wealthchart_title)

alt.FacetChart(...)